In [1]:
# Instalación del corpus.
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('cess_esp')

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

In [2]:
from nltk.corpus import cess_esp

sentences = cess_esp.sents()
words = cess_esp.words()
tags_words = cess_esp.tagged_words(fileids=None, tagset=None)

# Vemos las oraciones, palabras y etiquetas.
print("Oraciones:")
for sent in sentences[:5]:
    print(' '.join(sent))

print("\nPalabras:")
for word in words[:10]:
    print(word)

print("\nEtiquetas:")
for tag in tags_words[:10]:
    print(tag)

Oraciones:
El grupo estatal Electricité_de_France -Fpa- EDF -Fpt- anunció hoy , jueves , la compra del 51_por_ciento de la empresa mexicana Electricidad_Águila_de_Altamira -Fpa- EAA -Fpt- , creada por el japonés Mitsubishi_Corporation para poner_en_marcha una central de gas de 495 megavatios .
Una portavoz de EDF explicó a EFE que el proyecto para la construcción de Altamira_2 , al norte de Tampico , prevé la utilización de gas natural como combustible principal en una central de ciclo combinado que debe empezar a funcionar en mayo_del_2002 .
La electricidad producida pasará a la red eléctrica pública de México en_virtud_de un acuerdo de venta de energía de EAA con la Comisión_Federal_de_Electricidad -Fpa- CFE -Fpt- por una duración de 25 años .
EDF , que no quiso revelar cuánto *0* pagó por su participación mayoritaria en EAA , intervendrá como asistente en la construcción de Altamira_2 y , posteriormente , *0* se encargará de explotarla como principal accionista .
EDF y Mitsubishi pa

In [3]:
# Conjunto de etiquetas que se mantendrán según la práctica: verbos (v) y signos de puntuación (F)
tags_permitidas = {'v', 'F'}

# Conjunto reducido de etiquetas
tags_reducidas = set()

# Filtramos las etiquetas según las condiciones
for word, tag in tags_words:
    if len(tag) == 2 and tag[0] not in tags_permitidas:
        tags_reducidas.add((word, tag))
    elif len(tag) == 1:
        tags_reducidas.add((word, tag))

# Eliminamos las tuplas vacías
tags_reducidas.discard((u'*0*',))

# Visualizamos el conjunto reducido
print(list(tags_reducidas)[:50])


[('al_contrario', 'rg'), ('69', 'Z'), ('alegremente', 'rg'), ('codo_a_codo', 'rg'), ('7-4', 'Z'), ('puntualmente', 'rg'), ('495', 'Z'), ('siquiera', 'rg'), ('15_de_marzo', 'W'), ('43_por_ciento', 'Zp'), ('0,92', 'Z'), ('no_tanto', 'rn'), ('64', 'Z'), ('A_pesar_de_que', 'cs'), ('7.943', 'Z'), ('pasado_día_9', 'W'), ('considerablemente', 'rg'), ('No_bien', 'cc'), ('1:31.826', 'Z'), ('incondicionalmente', 'rg'), ('día_30', 'W'), ('año_sesenta_y_uno_después_de_Cristo', 'W'), ('14.000', 'Z'), ('Die_Materie_ist_nicht_anders_-LSB-', 'X'), ('6,7', 'Z'), ('a_lo_sumo', 'rg'), ('pasablemente', 'rg'), ('111,28', 'Z'), ('07.15', 'Z'), ('día_28', 'W'), ('1985', 'W'), ('es_cierto', 'rg'), ('siempre_que', 'cs'), ('a_lo_mejor', 'rg'), ('16.049', 'Z'), ('día_28_de_junio', 'W'), ('A_ratos', 'rg'), ('336', 'Z'), ('0,9355', 'Z'), ('En_todo_caso', 'rg'), ('de_costumbre', 'rg'), ('17.1', 'Z'), ('13:30', 'Z'), ('pesetas', 'Zm'), ('peor', 'rg'), ('9_de_marzo', 'W'), ('15', 'Z'), ('poco_menos_que', 'rg'), ('de_

Vamos a verlo conviertiendo el resultado en Dataframe.

In [4]:
import pandas as pd
df_etiquetas_reducidas = pd.DataFrame(tags_reducidas, columns=['Palabra', 'Etiqueta'])
print(df_etiquetas_reducidas.head(50))

                                Palabra Etiqueta
0                          al_contrario       rg
1                                    69        Z
2                           alegremente       rg
3                           codo_a_codo       rg
4                                   7-4        Z
5                          puntualmente       rg
6                                   495        Z
7                              siquiera       rg
8                           15_de_marzo        W
9                         43_por_ciento       Zp
10                                 0,92        Z
11                             no_tanto       rn
12                                   64        Z
13                       A_pesar_de_que       cs
14                                7.943        Z
15                         pasado_día_9        W
16                    considerablemente       rg
17                              No_bien       cc
18                             1:31.826        Z
19                  

Dividimos el corpus conforme nos pide la practica 90% training 10% test.

In [5]:
from sklearn.model_selection import train_test_split

# Dividir el corpus en datos de entrenamiento y prueba
train_tags, test_tags = train_test_split(df_etiquetas_reducidas, test_size=0.1, random_state=123)

# Imprimir la cantidad de oraciones en cada conjunto
print("Cantidad de oraciones en el conjunto de train:", len(train_tags))
print("Dataframe del training:\n", train_tags.head(10))
print("Cantidad de oraciones en el conjunto de test:", len(test_tags))
print("Dataframe del test:\n", test_tags.head(10))

Cantidad de oraciones en el conjunto de train: 1957
Dataframe del training:
              Palabra Etiqueta
415            Hasta       rg
1833      al_parecer       rg
1400     ligeramente       rg
2172            1976        W
12                64        Z
356           110,94        Z
142            fuera       rg
2030        nada_más       rg
1164  diligentemente       rg
1797          min.88        Z
Cantidad de oraciones en el conjunto de test: 218
Dataframe del test:
                Palabra Etiqueta
1361         De_súbito       rg
120            Hoy_día       rg
1779   16_de_noviembre        W
504            445.079        Z
498            970.000        Z
546             min.94        Z
576   fotográficamente       rg
1108      sino_también       cc
1201         cinco_mil        Z
883     04/20/00-16/00        Z


###Etiquetador morfosintácticos HMM

In [6]:
from nltk.tag import HiddenMarkovModelTagger
import pandas as pd
# Al averiguar que el metodo HiddenMarkov solo le podemos informar una lista, convertimos el Dataframe a lista.
tags_train = list(train_tags[['Palabra', 'Etiqueta']].itertuples(index=False, name=None))
# Etiquetador HMM
hmm_tags = HiddenMarkovModelTagger.train([tags_train])

Evaluamos el modelo.

In [7]:
#Hacemos la conversión que habiamos hecho en el train.
tags_test = list(test_tags[['Palabra', 'Etiqueta']].itertuples(index=False, name=None))
#Dejamos la parte del test para predecir la etiqueta.
test_tags_eval = list(test_tags['Palabra'])
#Pasamos el modelo entrenado para predecir las etiquetas.
hmm_tagged_test = hmm_tags.tag(test_tags_eval)
#Cogemos los valores reales del test.
true_tags = list(test_tags['Etiqueta'])
#Asignamos las etiquetas predichas.
predicted_tags = [tag for _, tag in hmm_tagged_test]

Visualizamos el etiquetado:

In [8]:
# Crear un DataFrame con las palabras, etiquetas predichas y etiquetas reales
df_resultado = pd.DataFrame({'Palabra': test_tags_eval, 'Etiqueta Predicha': predicted_tags, 'Etiqueta Real': true_tags})

# Imprimir el DataFrame
print(df_resultado)

             Palabra Etiqueta Predicha Etiqueta Real
0          De_súbito                 I            rg
1            Hoy_día                 W            rg
2    16_de_noviembre                rg             W
3            445.079                 Z             Z
4            970.000                rg             Z
..               ...               ...           ...
213            mismo                rg            rg
214              Que                 Z            cs
215   3,2_por_ciento                rg            Zp
216           60.000                 Z             Z
217  individualmente                rg            rg

[218 rows x 3 columns]


Sacamos el accuracy.

In [9]:
#contador de aciertos predichos
contador_aciertos = sum(1 for i in range(len(true_tags)) if true_tags[i] == predicted_tags[i])
#total de valores reales
contador_real = len(true_tags)
#accuracy
hmm_accuracy = "{:.4%}".format((contador_aciertos / contador_real))
print(f"Precisión del etiquetador HMM: {hmm_accuracy}")

Precisión del etiquetador HMM: 31.6514%


### Etiquetador tnt

In [10]:
from sklearn.model_selection import train_test_split

# Dividir el corpus en datos de entrenamiento y prueba
train_tags2, test_tags2 = train_test_split(df_etiquetas_reducidas, test_size=0.1, random_state=123)

# Imprimir la cantidad de oraciones en cada conjunto
print("Cantidad de oraciones en el conjunto de train:", len(train_tags2))
print("Dataframe del training:\n", train_tags2.head(10))
print("Cantidad de oraciones en el conjunto de test:", len(test_tags2))
print("Dataframe del test:\n", test_tags2.head(10))

Cantidad de oraciones en el conjunto de train: 1957
Dataframe del training:
              Palabra Etiqueta
415            Hasta       rg
1833      al_parecer       rg
1400     ligeramente       rg
2172            1976        W
12                64        Z
356           110,94        Z
142            fuera       rg
2030        nada_más       rg
1164  diligentemente       rg
1797          min.88        Z
Cantidad de oraciones en el conjunto de test: 218
Dataframe del test:
                Palabra Etiqueta
1361         De_súbito       rg
120            Hoy_día       rg
1779   16_de_noviembre        W
504            445.079        Z
498            970.000        Z
546             min.94        Z
576   fotográficamente       rg
1108      sino_también       cc
1201         cinco_mil        Z
883     04/20/00-16/00        Z


In [11]:
from nltk.tag import tnt
tags_train2 = list(train_tags2[['Palabra', 'Etiqueta']].itertuples(index=False, name=None))
tags_test2 = list(test_tags2[['Palabra', 'Etiqueta']].itertuples(index=False, name=None))

Evaluamos el modelo

In [12]:
#Dejamos la parte del test para predecir la etiqueta.
test_tags_eval2 = list(test_tags2['Palabra'])
#Cogemos los valores reales del test.
true_tags2 = list(test_tags2['Etiqueta'])

In [13]:
# Create a TnT tagger instance
tnt_tags = tnt.TnT()
# Train the TnT tagger on your training data
tnt_tags.train([tags_train2])
accuracy = tnt_tags.accuracy([tags_test2])
print("Accuracy of TnT Tagging:", accuracy)

Accuracy of TnT Tagging: 0.0


In [14]:
# Create a TnT tagger instance
tnt_tags = tnt.TnT()

# Train the TnT tagger on your training data
tnt_tags.train([tags_train])

# Use the TnT tagger to tag the test data
predicted_tags = [tag for word, tag in tnt_tags.tag(test_tags_eval)]

# Compare the predicted tags with the true tags and calculate the accuracy
correct_tags = sum(1 for pred, true in zip(predicted_tags, true_tags) if pred == true)
accuracy = correct_tags / len(true_tags)
print("Accuracy of TnT Tagging:", accuracy)


Accuracy of TnT Tagging: 0.0


In [15]:
# Crear un DataFrame con las palabras, etiquetas predichas y etiquetas reales
df_resultado_tnt = pd.DataFrame({'Palabra': test_tags_eval, 'Etiqueta Predicha': predicted_tags, 'Etiqueta Real': true_tags})

# Imprimir el DataFrame
print(df_resultado_tnt)

             Palabra Etiqueta Predicha Etiqueta Real
0          De_súbito               Unk            rg
1            Hoy_día               Unk            rg
2    16_de_noviembre               Unk             W
3            445.079               Unk             Z
4            970.000               Unk             Z
..               ...               ...           ...
213            mismo               Unk            rg
214              Que               Unk            cs
215   3,2_por_ciento               Unk            Zp
216           60.000               Unk             Z
217  individualmente               Unk            rg

[218 rows x 3 columns]


In [16]:
from nltk.tag import tnt
from sklearn.model_selection import train_test_split
sents_tags = cess_esp.tagged_sents()
# Dividir el corpus en datos de entrenamiento y prueba
train_tags2, test_tags2 = train_test_split(sents_tags, test_size=0.1, random_state=123)
tnt_tagger = tnt.TnT()
tnt_tagger.train(train_tags2)
accuracy = "{:.4%}".format((tnt_tagger.evaluate(test_tags2)))
print("Accuracy of TnT Tagging:", accuracy)

<ipython-input-16-5fba939d4ea3>:8: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = "{:.4%}".format((tnt_tagger.evaluate(test_tags2)))


Accuracy of TnT Tagging: 89.0850%


**Punto 3**

In [17]:
from nltk.tag import tnt
from sklearn.model_selection import KFold

sents_tags2 = cess_esp.tagged_sents()

accuracies = []

kfold = KFold(n_splits=10, shuffle=True, random_state=123)

for train_index, test_index in kfold.split(sents_tags):
    # Dividir el corpus en datos de entrenamiento y prueba para el pliegue actual
    train_tags = [sents_tags[i] for i in train_index]
    test_tags = [sents_tags[i] for i in test_index]

    # Entrenar el etiquetador TnT
    tnt_tagger = tnt.TnT()
    tnt_tagger.train(train_tags)

    # Evaluar el rendimiento del etiquetador en los datos de prueba
    accuracy = tnt_tagger.evaluate(test_tags)
    accuracies.append(accuracy)


<ipython-input-17-289673467bc8>:20: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = tnt_tagger.evaluate(test_tags)


In [19]:
import statistics
mean_accuracy = sum(accuracies) / len(accuracies)
std_accuracy = statistics.stdev(accuracies)

print("Precisiones obtenidas en cada pliegue:", accuracies)
print("Precisión media: {:.2f}".format(mean_accuracy))
print("Desviación estándar: {:.2f}".format(std_accuracy))

Precisiones obtenidas en cada pliegue: [0.8908495729118715, 0.900768112933361, 0.8958070617906684, 0.8944160338860478, 0.8950523161049477, 0.8959294576027889, 0.8957894195671173, 0.8940898345153664, 0.895232248823409, 0.8932706031611778]
Precisión media: 0.90
Desviación estándar: 0.00
